In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
path='../data/'

users = pd.read_csv(path+'users.csv')
books = pd.read_csv(path+'books.csv')
ratings = pd.read_csv(path+'train_ratings.csv')

print('users shape: ', users.shape)
print('books shape: ', books.shape)
print('ratings shape: ', ratings.shape)

users shape:  (68092, 3)
books shape:  (149570, 10)
ratings shape:  (306795, 3)


In [3]:
def print_full(x):
    pd.set_option('display.max_rows', 500)
    print(x)
    pd.reset_option('display.max_rows')

## `location` 전처리

In [4]:
users['location'] = users['location'].apply(lambda x: x.replace(',,', ','))

In [5]:
users[['user_id', 'location']][users['location'].apply(lambda x: sum([s==',' for s in x])>2)]

,user_id,location
435,259223,"twin falls, idaho, idaho, usa"
762,14387,"coalville, leicestershire, england, united kin..."
940,38464,"tharston, norwich, england, united kingdom"
1044,53688,"paris, france, ile de france, france"
1207,75842,"chicago, il, illinois, usa"
...,...,...
66162,179728,"brentwood bay, b.c., british columbia, canada"
66751,269037,"seattle, wa, washington, usa"
66770,202347,"seba beach, ab., alberta, canada"
67584,278215,"nelson, motueka, nz, new zealand"


In [6]:
# users[['user_id', 'location']][users['location'].apply(lambda x: sum([s==',' for s in x])>2)].to_csv('../msk/location_out.csv', index=False)


In [7]:
location_out_corrected = pd.read_csv('../msk/location_out copy.csv')


In [8]:
any(location_out_corrected['location'].apply(lambda x: sum([s==',' for s in x])>2))

False

In [9]:
users.loc[users['user_id'].isin(location_out_corrected['user_id']),'location'] = location_out_corrected['location'].values

In [10]:
any(users['location'].apply(lambda x: sum([s==',' for s in list(str(x))])>2))

False

### location -> location_city, location_state, location_country


In [11]:
users['location'] = users['location'].str.replace(r'[^0-9a-zA-Z:,]', '')

users['location_city'] = users['location'].apply(lambda x: x.split(',')[0].strip())
users['location_state'] = users['location'].apply(lambda x: x.split(',')[1].strip())
users['location_country'] = users['location'].apply(lambda x: x.split(',')[2].strip())

users = users.replace('na', np.nan)
users = users.replace('', np.nan)


/tmp/ipykernel_118709/2475927196.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  users['location'] = users['location'].str.replace(r'[^0-9a-zA-Z:,]', '')


In [12]:
users.nunique()

user_id             68092
location            17865
age                    91
location_city       11972
location_state       1457
location_country      219
dtype: int64

### location_country

In [13]:
# pd.set_option('display.max_rows', 500)
# print(users['location_country'].value_counts().sort_index())
# pd.reset_option('display.max_rows')
# unitedsates                           1
# unitedstaes                           1
# unitedstate                           1
# unitedstates                          9
# us, unitedstatesofamerica, losestadosunidosdenorteamerica -> usa
# usacurrentlylivinginengland
# usacanada
# america
# disgruntledstatesofamerica

# phillipines -> philippines
# uae, ua, unitedarabemirates
# nz -> newzealand
# c
# unitedkindgonm, uk, england -> unitedkingdom
# unknown

In [14]:
# -> usa
users.loc[users['location_country'].isin(['unitedsates', 'unitedstaes', 'unitedstate', 'unitedstates', 'us', 'unitedstatesofamerica', 'losestadosunidosdenorteamerica', 'usacurrentlylivinginengland', 'usacanada', 'america', 'disgruntledstatesofamerica', 'ua']),'location_country'] = 'usa'

In [15]:
# phillipines -> philippines
users.loc[users['location_country'].isin(['phillipines']),'location_country'] = 'philippines'


In [16]:
# uae, ua, unitedarabemirates
users.loc[users['location_country'].isin(['uae']),'location_country'] = 'unitedarabemirates'

# nz -> newzealand
users.loc[users['location_country'].isin(['nz']),'location_country'] = 'newzealand'

# c
users.loc[users['location_country'].isin(['c']),'location_country'] = np.nan
users.loc[users['location_state'].isin(['c']),'location_country'] = np.nan
users.loc[users['location_city'].isin(['c']),'location_country'] = np.nan

# unitedkindgonm, uk, england -> unitedkingdom
users.loc[users['location_country'].isin(['unitedkindgonm', 'uk', 'england']),'location_country'] = 'unitedkingdom'

# unknown
users.loc[users['location_country'].isin(['unknown']),'location_country'] = np.nan
users.loc[users['location_state'].isin(['unknown']),'location_country'] = np.nan
users.loc[users['location_city'].isin(['unknown']),'location_country'] = np.nan

### location_state 전처리

In [17]:
state_names = {
 'AK': 'Alaska',
 'AL': 'Alabama',
 'AR': 'Arkansas',
 'AS': 'American Samoa',
 'AZ': 'Arizona',
 'CA': 'California',
 'CO': 'Colorado',
 'CT': 'Connecticut',
 'DC': 'District of Columbia',
 'DE': 'Delaware',
 'FL': 'Florida',
 'GA': 'Georgia',
 'GU': 'Guam',
 'HI': 'Hawaii',
 'IA': 'Iowa',
 'ID': 'Idaho',
 'IL': 'Illinois',
 'IN': 'Indiana',
 'KS': 'Kansas',
 'KY': 'Kentucky',
 'LA': 'Louisiana',
 'MA': 'Massachusetts',
 'MD': 'Maryland',
 'ME': 'Maine',
 'MI': 'Michigan',
 'MN': 'Minnesota',
 'MO': 'Missouri',
 'MP': 'Northern Mariana Islands',
 'MS': 'Mississippi',
 'MT': 'Montana',
 'NC': 'North Carolina',
 'ND': 'North Dakota',
 'NE': 'Nebraska',
 'NH': 'New Hampshire',
 'NJ': 'New Jersey',
 'NM': 'New Mexico',
 'NV': 'Nevada',
 'NY': 'New York',
 'OH': 'Ohio',
 'OK': 'Oklahoma',
 'OR': 'Oregon',
 'PA': 'Pennsylvania',
 'PR': 'Puerto Rico',
 'RI': 'Rhode Island',
 'SC': 'South Carolina',
 'SD': 'South Dakota',
 'TN': 'Tennessee',
 'TX': 'Texas',
 'UT': 'Utah',
 'VA': 'Virginia',
 'VI': 'Virgin Islands',
 'VT': 'Vermont',
 'WA': 'Washington',
 'WI': 'Wisconsin',
 'WV': 'West Virginia',
 'WY': 'Wyoming',
 }
state_names_lower = {}
for k, v in state_names.items():
    state_names_lower[k.lower()] = v.lower().replace(' ', '')
users[(users['location_country']=='usa')&(users['location_state'].isin(state_names_lower.keys()))][['location_country', 'location_state']]


,location_country,location_state
54,usa,ga
503,usa,dc
781,usa,dc
834,usa,nj
1064,usa,nm
...,...,...
66218,usa,dc
67264,usa,ne
67333,usa,dc
67372,usa,ca


In [18]:
users.loc[(users['location_country']=='usa')&(users['location_state'].isin(state_names_lower.keys())), 'location_state'] = users[(users['location_country']=='usa')&(users['location_state'].isin(state_names_lower.keys()))]['location_state'].map(state_names_lower)

### 존재하는 city 정보로 state, country 결측 대치

In [19]:
users['location_pp'] = users[['location_city', 'location_state', 'location_country']].apply(lambda x: ', '.join(x.fillna('n/a').values), axis=1).values
users[['location_city_pp', 'location_state_pp', 'location_country_pp']] = users[['location_city', 'location_state', 'location_country']].copy()

In [20]:
users.isna().sum()

user_id                    0
location                   0
age                    27833
location_city            124
location_state          3212
location_country        2103
location_pp                0
location_city_pp         124
location_state_pp       3212
location_country_pp     2103
dtype: int64

In [21]:
# location_state 결측 대치
modify_location = users[((users['location_state_pp'].isna()))&(users['location_city_pp'].notnull())]['location_city_pp'].values
location_list = []
for location in modify_location:
    try:
        tmp_users = users[users['location_city_pp'].notnull()]
        right_location = tmp_users[(tmp_users['location_city_pp']==location)&((tmp_users['location_state_pp'].notnull()))]['location_pp'].value_counts().index[0]
        location_list.append(right_location)
    except:
        pass
for location in location_list:
    users.loc[(users['location_city_pp']==location.split(',')[0])&(users['location_state_pp'].isnull()),'location_state_pp'] = location.split(',')[1].strip()
    

# users = users.drop(['location'], axis=1)

In [22]:
# location_country 결측 대치
modify_location = users[((users['location_country_pp'].isna()))&(users['location_city_pp'].notnull())]['location_city_pp'].values
location_list = []
for location in modify_location:
    try:
        tmp_users = users[users['location_city_pp'].notnull()]
        right_location = tmp_users[(tmp_users['location_city_pp']==location)&(tmp_users['location_country_pp'].notnull())]['location_pp'].value_counts().index[0]
        location_list.append(right_location)
    except:
        pass
for location in location_list:
    users.loc[(users['location_city_pp']==location.split(',')[0])&(users['location_country_pp'].isnull()),'location_country_pp'] = location.split(',')[2].strip()
    

# users = users.drop(['location'], axis=1)

In [23]:
# modify_location = users[(users['location_country_pp'].isna())&(users['location_city_pp'].notnull())]['location_city_pp'].values

# location_list = []
# for location in modify_location:
#     try:
#         right_location = users[(users['location_pp'].str.contains(location))&(users['location_country_pp'].notnull())]['location_pp'].value_counts().index[0]
#         location_list.append(right_location)
#     except:
#         pass
# for location in location_list:
#     users.loc[users[users['location_city_pp']==location.split(',')[0]].index,'location_state_pp'] = location.split(',')[1].strip()
#     users.loc[users[users['location_city_pp']==location.split(',')[0]].index,'location_country_pp'] = location.split(',')[2].strip()

In [24]:
users['location_state_pp'] = users['location_state_pp'].replace('n/a', np.nan)
users['location_country_pp'] = users['location_country_pp'].replace('n/a', np.nan)

In [25]:
users.isna().sum()

user_id                    0
location                   0
age                    27833
location_city            124
location_state          3212
location_country        2103
location_pp                0
location_city_pp         124
location_state_pp        469
location_country_pp      172
dtype: int64

### state, city 최빈값 대치

In [26]:
# state 최빈값 대치
state_mode = users.groupby(['location_country_pp'])['location_country_pp'].agg(pd.Series.mode)

idx = users[(users['location_state_pp'].isna())].index
for i in idx:
    try:
        tmp_country = users.loc[i, 'location_country_pp']
        if isinstance(state_mode[tmp_country], str):
            users.loc[i, 'location_state_pp'] = state_mode[tmp_country]
        else:
            users.loc[i, 'location_state_pp'] = state_mode[tmp_country][0]
    except:
        pass
        

In [27]:
# city 최빈값 대치
city_mode1 = users.groupby(['location_country_pp','location_state_pp'])['location_city_pp'].agg(pd.Series.mode)
city_mode2 = users.groupby(['location_state_pp'])['location_city_pp'].agg(pd.Series.mode)
city_mode3 = users.groupby(['location_country_pp'])['location_city_pp'].agg(pd.Series.mode)

idx = users[(users['location_city_pp'].isna())].index
for i in idx:
    tmp_state = users.loc[i, 'location_state_pp']
    tmp_country = users.loc[i, 'location_country_pp']
    try:
        if isinstance(city_mode1[tmp_country,tmp_state], str):
            users.loc[i, 'location_city_pp'] = city_mode1[tmp_country, tmp_state]
        else:
            users.loc[i, 'location_city_pp'] = city_mode1[tmp_country, tmp_state][0]
    except:
        try:
            if isinstance(city_mode1[tmp_country,tmp_state], str):
                users.loc[i, 'location_city_pp'] = city_mode2[tmp_state]
            else:
                users.loc[i, 'location_city_pp'] = city_mode2[tmp_state][0]
        except:
            try:
                if isinstance(city_mode1[tmp_country,tmp_state], str):
                    users.loc[i, 'location_city_pp'] = city_mode3[tmp_country]
                else:
                    users.loc[i, 'location_city_pp'] = city_mode3[tmp_country][0]
            except:
                pass

In [28]:
users.isna().sum()

user_id                    0
location                   0
age                    27833
location_city            124
location_state          3212
location_country        2103
location_pp                0
location_city_pp          15
location_state_pp        117
location_country_pp      172
dtype: int64

### location_state, location_city 
- 각 `location_country` 기준
- `location_state`의 nunique < 15 -> `location_country` + "_others"
- `location_city`의 nunique < 10 -> `location_state` + "_others"

In [29]:
countries = users['location_country_pp'].unique()
for country in countries:
    tmp = users[users['location_country_pp']==country]['location_state_pp'].value_counts()
    try:
        users.loc[(users['location_country_pp']==country)&(users['location_state_pp'].isin(tmp[tmp<15].index)), 'location_state_pp'] = country+'_others'
    except:
        pass
    
    states = users[(users['location_country_pp']==country)]['location_state_pp'].unique()
    for state in states:
        tmp = users[(users['location_country_pp']==country)&(users['location_state_pp']==state)]['location_city_pp'].value_counts()
        try:
            users.loc[(users['location_country_pp']==country)&(users['location_state_pp']==state)&(users['location_state_pp'].isin(tmp[tmp<10].index)), 'location_city_pp'] = state+'_others'
        except:
            pass

In [30]:
users

,user_id,location,age,location_city,location_state,location_country,location_pp,location_city_pp,location_state_pp,location_country_pp
0,8,"timmins,ontario,canada",NaN,timmins,ontario,canada,"timmins, ontario, canada",ontario_others,ontario,canada
1,11400,"ottawa,ontario,canada",49.0,ottawa,ontario,canada,"ottawa, ontario, canada",ontario_others,ontario,canada
2,11676,"na,na,na",NaN,NaN,NaN,NaN,"n/a, n/a, n/a",NaN,NaN,NaN
3,67544,"toronto,ontario,canada",30.0,toronto,ontario,canada,"toronto, ontario, canada",ontario_others,ontario,canada
4,85526,"victoria,britishcolumbia,canada",36.0,victoria,britishcolumbia,canada,"victoria, britishcolumbia, canada",victoria,britishcolumbia,canada
...,...,...,...,...,...,...,...,...,...,...
68087,278376,"danville,pennsylvania,usa",54.0,danville,pennsylvania,usa,"danville, pennsylvania, usa",danville,pennsylvania,usa
68088,278621,"victoria,delaware,canada",74.0,victoria,delaware,canada,"victoria, delaware, canada",victoria,canada_others,canada
68089,278636,"irvington,alabama,usa",NaN,irvington,alabama,usa,"irvington, alabama, usa",irvington,alabama,usa
68090,278659,"vancouver,washington,usa",33.0,vancouver,washington,usa,"vancouver, washington, usa",vancouver,washington,usa


In [31]:
users.isna().sum()

user_id                    0
location                   0
age                    27833
location_city            124
location_state          3212
location_country        2103
location_pp                0
location_city_pp          15
location_state_pp        117
location_country_pp      172
dtype: int64

In [40]:
users.nunique()

user_id             68092
age                   496
location_city       11169
location_state        416
location_country      199
dtype: int64

In [32]:
users[(users.location_city_pp.isna())&(users.location_state_pp.isna())&(users.location_country_pp.isna())]

,user_id,location,age,location_city,location_state,location_country,location_pp,location_city_pp,location_state_pp,location_country_pp
2,11676,"na,na,na",NaN,NaN,NaN,NaN,"n/a, n/a, n/a",NaN,NaN,NaN
1679,146804,"na,na,",NaN,NaN,NaN,NaN,"n/a, n/a, n/a",NaN,NaN,NaN
7760,70063,"na,na,na",NaN,NaN,NaN,NaN,"n/a, n/a, n/a",NaN,NaN,NaN
8634,112345,",,",39.0,NaN,NaN,NaN,"n/a, n/a, n/a",NaN,NaN,NaN
38772,156948,"na,na,",NaN,NaN,NaN,NaN,"n/a, n/a, n/a",NaN,NaN,NaN
48206,169489,"na,,",NaN,NaN,NaN,NaN,"n/a, n/a, n/a",NaN,NaN,NaN
58435,56072,",,",NaN,NaN,NaN,NaN,"n/a, n/a, n/a",NaN,NaN,NaN
60682,64582,"na,na,",30.0,NaN,NaN,NaN,"n/a, n/a, n/a",NaN,NaN,NaN
67104,218092,"na,na,",NaN,NaN,NaN,NaN,"n/a, n/a, n/a",NaN,NaN,NaN


In [33]:
# print_full(users.loc[(users.location_city_pp.isna())|(users.location_state_pp.isna())|(users.location_country_pp.isna()), ['location_city_pp', 'location_state_pp','location_country_pp']])

In [34]:
tmp_mode = users.groupby('location_city_pp')['age'].agg(lambda x: pd.Series.mode(x).mean())

In [35]:
for i in tmp_mode.index:
    users.loc[(users['location_city_pp']==i)&(users['age']).isna(),'age'] = tmp_mode[i]

In [36]:
users.isna().sum()

user_id                   0
location                  0
age                    3124
location_city           124
location_state         3212
location_country       2103
location_pp               0
location_city_pp         15
location_state_pp       117
location_country_pp     172
dtype: int64

In [37]:
users=users[['user_id','age','location_city_pp','location_state_pp','location_country_pp']]
users.columns=['user_id','age','location_city','location_state','location_country']

In [38]:
users.head()

,user_id,age,location_city,location_state,location_country
0,8,30.0,ontario_others,ontario,canada
1,11400,49.0,ontario_others,ontario,canada
2,11676,NaN,NaN,NaN,NaN
3,67544,30.0,ontario_others,ontario,canada
4,85526,36.0,victoria,britishcolumbia,canada


In [39]:
users.to_csv('../data/users_preprocessed.csv', index=False)